## Outline
 
- how things can go wrong
- recognizing error conditions and their causes
- dealing with errors
- how to determine if you have a bad ASDF file

This tutorial is more of a checklist than a real tutorial, but it should be a useful reference

### Important Background Material

One of the key aspects of ASDF are the tools it provides to validate that the ASDF
file is correct. How does an ASDF library use these tools to know what is correct?
ASDF files can be associated with files that define expectations for the file. 
Besides the obvious requirement that the yaml be correctly formatted yaml, ASDF
makes use of associated files called "schema" files and the ASDF library uses these to
check the validity of a file, either in reading from or in writing to an ASDF file. 
Normally these expectations are provided by the tags a file contains (but currently
JWST uses a different approach). Usually tags have corresponding "schema" files.

What exactly is a schema file? It is a yaml-formatted text file 
that contains rules for what is expected of the content associated with an identification tag. It also contains what attributes are expected and the
types of values they are expected to have (these types may themselves be other tags).
This tutorial will not go into the details of these schemas (how to interpret and 
write schemas for custom content will be the subject of a different tutorial).
If the library encounters a file that violates these rules, an error will occur.

This checking is called validation and it is by default, performed when reading
ASDF files, as well as writing content to them. Validation can significantly 
affect the performance of reading and writing ASDF files, so it can be turned off
in either case.

### How errors occur

There are a number of ways errors or warnings may arise. Here are common reasons:

- The needed library to deal with a tag is not installed. This usually results in a
  warning (depending on the read options selected). When a tag is encounterd that
  does not have a registered library to deal with it, the content is generally 
  read in as basic Python dicts and lists, but no conversion to a special Python
  object is done.
- The version of the tag is not supported by the version of the library. This 
  also usually results in a warning, and no conversion to an object is 
  performed.
- The content of the file does not satisfy the rules of the schema file associated
  with a particular tag.
- The yaml itself has syntax errors. This normally should not occur when the file
  is generated by the ASDF library, but if someone has edited the yaml content
  by hand, they may have introduced syntax errors.
- Or, our favorite, there is a bug in the library.


### Diagnosing errors

If you get errors while validiation is off, **the very first thing you should do 
is load (or write) the file with validation on.** The resulting errors when that 
is not turned on may be very mysterious and not indicate the root problem.

---

**AsdfConversionWarning:** `tag:somewhere.org:some/missing/tag-1.0.0 is not recognized, converting to raw Python data structure`

Missing extension support on read.  Can be solved by installing a Python package containing the missing extension.  Can be safely ignored if the user does not need the object.

---

**AsdfWarning:** `File 'file:///path/to/some_file.asdf' was created with extension 'some.Extension' from package foo-2.0.0, but older version foo-1.5.0 is installed`

The file was (possibly) created using a newer version of an extension.  May not actually cause problems.

---

**RepresenterError:** `('cannot represent an object', SomeUnsupportedClass())`

Missing extension support on write.  Can be solved by installing a Python package containing the missing extension, or by removing the offending object from the tree.

---

**ValidationError:** `'version' is a required property`

(and various other validation failures)

The tree content failed validation against a schema.  Can occur on read or write.

---

**AsdfWarning:** `File 'file:///path/to/some_file.asdf' was created with extension class 'some.missing.Extension', which is not currently installed`

The file was created using an extension that isn't installed on the current system.  Probably accompanied by the "tag is not recognized" warning above.

---

**ValueError:** `ASDF Standard version 9.12.0 is not supported by asdf==2.7.1.  Available ASDF Standard versions: 1.0.0, 1.1.0, 1.2.0, 1.3.0, 1.4.0, 1.5.0`

The file was written under an ASDF Standard version that isn't supported by this version of asdf.  Can be solved by upgrading the asdf package.

---

**AsdfConversionWarning:** `Version 14.8.5 of tag:somewhere.org/some/tag is not compatible with any existing tag implementations`

Missing support for a specific version of a tag on read.  Probably can be solved by updating the package that provides support for the tag.

---

**YAML syntax errors:** These may appear as a variety of errors, but currently they are not ASDF errors. The plan is to have ASDF wrap these as some sort of ASDFYamlSyntaxError to make it clearer.